### This can be the final functionized version, where running the tests is easy - just change 'n' and 'bound'
### It is fast at sieving, but is not factoring properly - can you take a look at anything that looks off? 
#### Need to only check after B smooth numbers have already been generated

In [1]:
import math
import numpy as np
import sympy
import time 

In [2]:
def calc_time(start, end):
    seconds = end - start
    mins = seconds / 60
    print('Time in seconds: ', str(seconds), ', Time in minutes: ', str(mins))
    return seconds, mins

In [3]:
# Final optimized version 
def get_primes_less_thanB(bound):                      # eratosthenes way
    check_until = int(bound**0.5)
    unmarked = np.array([x for x in range(2, bound)])
    q = 2
    i = 0
    while True:
        u = unmarked[i]
        if u> check_until:               # only check numbers up to sqrt(bound)
            break
        u_idx = np.where(unmarked == u)[0][0]
        max_idx = bound//u
        listof_multiples = [u*i for i in range(q, max_idx+1)]
        u_multiples = np.array(listof_multiples)
        unmarked = np.setdiff1d(unmarked, u_multiples)   # numpy set diff without changing order of elements in list
        q = u
        i +=1
    print("Number of elements less than bound = ", bound, " is: ", len(unmarked))
    return (unmarked)

In [4]:
# Correct optimized version
def check_Bsmooth(primes, num):
    powers_of_primes= []
    num_remaining= num                         
    for prime in primes: 
        i = 0                                   # i = power of prime
        if (num_remaining%prime == 0):
            while (num_remaining%prime ==0):   #keep dividing for each power of the prime
                quotient = num_remaining/prime
                num_remaining = quotient
                i +=1
        powers_of_primes.append(i)
        if num_remaining == 1:     
            diff = len(primes)-len(powers_of_primes)
            if diff>0:
                zeros = [0 for k in range(diff)]
                return powers_of_primes + zeros
            return powers_of_primes
    #print("Number is not B smooth")
    return []

In [5]:
# Correct optimized version
def check_Bsmooth(primes, num):
    powers_of_primes= []
    num_remaining= num                         
    for prime in primes: 
        i = 0                                   # i = power of prime
        if (num_remaining%prime == 0):
            while (num_remaining%prime ==0):   #keep dividing for each power of the prime
                quotient = num_remaining/prime
                num_remaining = quotient
                i +=1
        powers_of_primes.append(i)
        if num_remaining == 1:     
            diff = len(primes)-len(powers_of_primes)
            if diff>0:
                zeros = [0 for k in range(diff)]
                return powers_of_primes + zeros
            return powers_of_primes
    #print("Number is not B smooth")
    return []

In [6]:
def a_congruent_b(a, b, n):
    if a==(b%n):
        return True
    return False

In [7]:
def generate_BSsquares(primes, n, multiple):
    exponents_list = []             # Add other fancy ways of finding candidates for xsquare
    xsquares = []           # the x
    x_vals = []
    residues = []           # the y
    iters = 0
    while(True):
        if iters>100000000:
            break
        x = int(math.sqrt(n*multiple)) +1
        xsquare = x**2
        residue = xsquare%n
        exp_vec = check_Bsmooth(primes, residue)
        if exp_vec != []:
            residues.append(residue)   # x^2
            xsquares.append(xsquare)
            exponents_list.append(exp_vec)
            x_vals.append(x)
            print("Found B-smooth square! with exponents: ", exp_vec, " and quadratic residue: ", residue)
        if len(residues) > len(primes):
            print("Breaking, len of residues: ", len(residues), " len of primes: ", len(primes))
            break
        multiple += 1
        iters += 1
    return [exponents_list, residues, iters, xsquares, x_vals]

#### Choose the bound value

In [8]:
bound = 20
primes = np.array(get_primes_less_thanB(bound))

Number of elements less than bound =  20  is:  8


#### Test: Choose an N of your liking (8-15 digits)

In [9]:
# A test on a small prime!
# n = 10710336959293
# n = 95775679     
n = 143
multiple = 4
start = time.time()

results = generate_BSsquares(primes, n, multiple)

end = time.time()
calc_time(start, end)
print("iterations, ", results[2])
print("B smooth numbers found: ", results[1], "\ntotal: ", len(results[1]))

Found B-smooth square! with exponents:  [2, 0, 0, 0, 0, 0, 0, 0]  and quadratic residue:  4
Found B-smooth square! with exponents:  [1, 0, 0, 1, 0, 0, 0, 0]  and quadratic residue:  14
Found B-smooth square! with exponents:  [1, 1, 0, 1, 0, 0, 0, 0]  and quadratic residue:  42
Found B-smooth square! with exponents:  [2, 1, 0, 0, 0, 0, 0, 0]  and quadratic residue:  12
Found B-smooth square! with exponents:  [0, 2, 0, 0, 0, 0, 0, 0]  and quadratic residue:  9
Found B-smooth square! with exponents:  [1, 0, 0, 1, 0, 0, 0, 0]  and quadratic residue:  14
Found B-smooth square! with exponents:  [0, 3, 0, 0, 0, 0, 0, 0]  and quadratic residue:  27
Found B-smooth square! with exponents:  [4, 1, 0, 0, 0, 0, 0, 0]  and quadratic residue:  48
Found B-smooth square! with exponents:  [0, 0, 0, 1, 1, 0, 0, 0]  and quadratic residue:  77
Breaking, len of residues:  9  len of primes:  8
Time in seconds:  0.0044651031494140625 , Time in minutes:  7.44183858235677e-05
iterations,  9
B smooth numbers fou

### The following uses (results, n) produced from last run cell 

In [10]:
def drop_zero_rows(A): 
    return A[~np.all(A == 0, axis=1)]

In [11]:
from sympy import *          # Start using np arrays

A = np.array(results[0])      # results = [exponents_list, residues, iters, xsquares]
store_pows = A.copy()
A = (np.transpose(A))%2
print(A)

[[0 1 1 0 0 1 0 0 0]
 [0 0 1 1 0 0 1 1 0]
 [0 0 0 0 0 0 0 0 0]
 [0 1 1 0 0 1 0 0 1]
 [0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]]


In [12]:
col_num = len(A[0])
row_num = len(A)
print("Old num rows and num columns: ", row_num, col_num)
L = Matrix(A)

L = np.array(L.rref()[0])      # reduced row echelon form
L2  = drop_zero_rows(L)        # get rid of primes we don't have to worry about in general
L2 = L2 %2

print("Reduced row echelon form AND mod 2 is: ")
print(L2)

print("New num rows, num columns: ", len(L2), len(L2[0]))

Old num rows and num columns:  8 9
Reduced row echelon form AND mod 2 is: 
[[0 1 0 1 0 1 1 1 0]
 [0 0 1 1 0 0 1 1 0]
 [0 0 0 0 0 0 0 0 1]]
New num rows, num columns:  3 9


In [21]:
# Choose a subset of columns (equations) to be 1 - those with 0 in pivot or 1 but the rowsum adds to 0mod2
def getA_solution(reduced_matrix):
    L = reduced_matrix
    col_num = len(L[0])
    row_num = len(L)
    v = np.ones(col_num, dtype = 'int')
    for i in reversed(range(row_num)):
        for j in range(col_num):
            if L[i][j] ==1:
                if (L[i].sum()%2) !=0:                     # the exponents for this prime doesn't add up to zero
                    v[j] = 0                  # can't use this equation
                    break     # no need to  check cells above, as reduced echelon form
    return(v)


In [22]:
v = getA_solution(L2)     # tells you which equations we can choose
print(v)

[1 0 1 1 1 1 1 1 0]


In [23]:
def get_root_modN(square, n): # xsq is the number left after multiplying all RELEVANT x squares found, check next func
    result = int(math.sqrt(square))%n
    return result

In [24]:
def get_product(v, num_list):  # just multiply all RELEVANT (depends on v) numbers in a list together
    product= 1       
    for i in range(len(v)):
        if v[i] ==0:
            continue
        product = product * num_list[i]
#         product = product%n    # not reducing mod n yet
    return product

In [25]:
# results is of form: [exponents_list, residues, iters, xsquares, x_vals (x before squaring)]
residues = results[1]    # x%n, RHS
xsquares = results[3]

ysq = get_product(v, residues) 
xsq = get_product(v, xsquares)

print("final x squared mod n: ", xsq%n, " final y squared mod n: ", ysq%n) # just a check - they have to be equal

final x squared mod n:  3  final y squared mod n:  3


In [26]:
x_vals = results[4]
x = get_product(v, x_vals)%n
y = get_root_modN(ysq, n)%n

if a_congruent_b(x, y, n):
    print("Oh no, x congruent y mod n!")
else:
    print("\n")
    print("Found x not congruent y mod n! A Factor!!")  # THIS ISN'T TRUE! GCD(x-y)doesn't return the right value :"(
    print(x, y)



Found x not congruent y mod n! A Factor!!
61 126


In [27]:
diff = abs(x-y)
print("x-y is: ", diff)
gcd = math.gcd(diff, n)
print("GCD found is: ", gcd)
print("other factor of n is: ", int(n/gcd)) 

x-y is:  65
GCD found is:  13
other factor of n is:  11


### THE END